<a href="https://colab.research.google.com/github/HandersThe/Capstone_Project_ML/blob/master/ProjectCapstone_EfficientNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Dependencies
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetV2B0

In [ ]:
#Install Kaggle
!pip install -q kaggle

You can get Kaggle api by creating an account from their website

In [ ]:
#Upload kaggle api
from google.colab import files
files.upload()

In [ ]:
#Create directory for kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#Download dataset and unzip
!kaggle datasets download -d techsash/waste-classification-data
!unzip waste-classification-data.zip

In [ ]:
#The number of output classes
NUM_CLASSES = 2

#Dataset directory path
train_dir = "dataset/DATASET/TRAIN/"
valid_dir = "dataset/DATASET/TEST/"

#Init for dataset
batch_size = 32
img_height = 160
img_width = 160

In [ ]:
#Training dataset
train_ds = keras.utils.image_dataset_from_directory(train_dir,
                                                    shuffle=True,
                                                    image_size=(img_height, img_width),
                                                    batch_size=batch_size)

#Validation dataset
val_ds = keras.utils.image_dataset_from_directory(valid_dir,
                                                  shuffle=True,
                                                  image_size=(img_height, img_width),
                                                  batch_size=batch_size)

In [ ]:
#Visualise the data
class_names = ["Organic", "Inorganic"]
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
#Creating a Test dataset
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 5)
val_ds = val_ds.skip(val_batches // 5)

In [ ]:
#Configure dataset for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
#Callback (use this if you want to increase the number of epochs or make model training automatically stop)
#The defaults are 0.95 for accuracy and 0.93 for val_accuracy
class myCallback(keras.callbacks.Callback):
      def on_epoch_end(self,epoch,logs={}):
        if((logs.get('accuracy')>0.95) and (logs.get('val_accuracy')>0.93)):
          self.model.stop_training = True

In [ ]:
#Data augmentation
data_augmentation = Sequential([
  layers.RandomRotation(factor=0.15),
  layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
  layers.RandomFlip(),
  layers.RandomContrast(factor=0.1),
])

#Preprocess Input
preprocess_input = keras.applications.efficientnet_v2.preprocess_input

In [ ]:
#Model build
inputs = keras.Input(shape=(img_height, img_width, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)

#Base Model
base_model = EfficientNetV2B0(input_tensor=x,
                              include_top=False,
                              weights="imagenet")

#Freeze the pretrained weights
base_model.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(2)(x)
model = keras.Model(inputs, outputs)

#Model compile
model.compile(optimizer="adam",
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
#Model summary
model.summary()

In [ ]:
#Initial model evaluation
loss0, accuracy0 = model.evaluate(val_ds)

In [ ]:
#Training epochs
initial_epochs=3

#Model fit
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=initial_epochs
)

In [ ]:
#Visualise training results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
#Fine tuning
#Take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

#Unfreeze the top 20 layers while leaving BatchNorm layers frozen
for layer in base_model.layers[-20:]:
  if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

##Or you can use this
#Fine-tune from this layer onwards
#fine_tune_at = 100

#Freeze all the layers before the `fine_tune_at` layer
#for layer in base_model.layers[:fine_tune_at]:
#  layer.trainable = False

#Fine tuning model compile
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#Fine tuning model summary
model.summary()

In [ ]:
#Fine tune epochs
fine_tune_epochs = 3
total_epochs =  initial_epochs + fine_tune_epochs

#Fine tuning model fit
history_fine = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=total_epochs
)

In [ ]:
#Visualise fine tunned model
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
#Test dataset model evaluation
loss, accuracy = model.evaluate(test_ds)
print('Test accuracy :', accuracy)

In [ ]:
#Predict one uploaded images
uploaded = files.upload()

for fn in uploaded.keys():

  path = fn
  img = tf.keras.utils.load_img(path,
                                target_size=(img_height, img_width)
  )
  imgplot = plt.imshow(img)
  x = tf.keras.utils.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  logits = model.predict(images)
  softmax = tf.nn.softmax(logits[0])


  print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(softmax)], 100 * np.max(softmax))
  )

In [ ]:
#Model save directory
model_save_location = "/content/Model/TrashSort"

In [ ]:
#Save Keras model
model.save(model_save_location)

In [ ]:
#Optimise prediction for GCP Vertex AI (don't use this if you want to use TF Lite)
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.uint8)
    resized = tf.image.resize(decoded, size=(img_height, img_width))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, img_height, img_width, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(model, model_save_location, signatures={"serving_default": serving_fn})

In [ ]:
#For android apps (This model has bigger size than the one in EfficientNetLite notebook)
#Converting to TF Lite
converter = tf.lite.TFLiteConverter.from_saved_model(model_save_location)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

#Writing TF Lite model
with open("model.tflite", "wb") as f:
    f.write(tflite_quant_model)

In [ ]:
#Download Model
!zip -r TrashSort.zip /content/Model/TrashSort

Please refer to https://codelabs.developers.google.com/vertex-image-deploy and https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_tf_serving_function.ipynb for setting up GCP Vertex AI enviroment